In [ ]:
import predictions_elsa as p
import numpy as np
import pandas as pd
import pickle

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

In [ ]:
with open("scaler.pkl", "rb") as f:
    pickle.load(f)

with open("model.pkl", "rb") as f:
    pickle.load(f)


In [ ]:
class inputUser(BaseModel):
    type : str
    subtype : str
    bedroomCount : float
    bathroomCount : float
    province : str
    locality : str
    postCode : int
    habitableSurface : float
    hasAttic : str
    hasBasement : str
    hasDressingRoom : bool
    hasDiningRoom : bool
    buildingCondition : str
    buildingConstructionYear : float
    facedeCount : float
    hasLift : bool
    floodZoneType : str
    heatingType : str
    hasHeatPump : bool
    hasPhotovoltaicPanels : bool
    hasThermicPanels : bool
    kitchenType : str
    landSurface : float
    hasLivingRoom : bool
    hasGarden : bool
    gardenSurface : float
    parkingCountIndoor : float
    parkingCountOutdoor : float
    hasAirConditioning : bool
    hasArmoredDoor : bool
    hasVisiophone : bool
    hasOffice : bool
    toiletCount : float
    hasSwimmingPool : bool
    hasFireplace : bool
    hasTerrace : bool
    terraceSurface : float
    epcScore : str

user_dict = {}

def user_input_preparation(user : inputUser):
    #id : float
    #url : str
    #roomCount : float
    #monthlyCost : float
    #diningRoomSurface : float
    #terraceOrientation : str
    #accessibleDisabledPeople : bool
    #floorCount : float
    #streetFacadeWidth : float
    #kitchenSurface : float
    #livingRoomSurface : float
    #hasBalcony : bool
    #gardenOrientation : str
    #price : float
    

In [13]:
app = FastAPI()

In [ ]:
def prediction(user_dict):
    # 1. Création du DataFrame à partir du dict 'propre' (raw features)
    df_new = pd.DataFrame([user_dict])

    # 2. Nettoyage & feature engineering
    df_new_clean = p.cleaning_dataframe(df_new, is_training=False)

    # 3. Imputations
    df_new_clean_imputed = p.transform_cleaning_traintestsplit(df_new_clean, p.stats_from_X_train, is_training=False)

    # 4. Scaling (en gardant le fit de tes données d’entraînement)
    columns_for_model = p.scaler.feature_names_in_.tolist()
    df_new_clean_imputed = df_new_clean_imputed[columns_for_model]

    df_new_scaled = p.scaler.transform(df_new_clean_imputed)

    # 5. Prédiction
    predicted_price = p.xgb_model.predict(df_new_scaled)

    return (f"Le prix prédit est {predicted_price[0]:,.0f} €".replace(',', ' '))

prediction(user_dict)

In [ ]:
df_final_order = ['id', 'bedroomCount', 'bathroomCount', 'habitableSurface', 'hasAttic',
       'hasBasement', 'hasDressingRoom', 'hasDiningRoom', 'hasLift',
       'hasHeatPump', 'hasPhotovoltaicPanels', 'hasThermicPanels',
       'landSurface', 'hasLivingRoom', 'gardenSurface', 'hasAirConditioning',
       'hasArmoredDoor', 'hasVisiophone', 'hasOffice', 'toiletCount',
       'hasSwimmingPool', 'hasFireplace', 'terraceSurface', 'price',
       'epc_enum', 'parkingCount', 'subtype_group',
       'buildingConstructionYear_mapping', 'isHouse', 'province_mapping',
       'buildingCondition_mapping', 'floodZoneType_mapping',
       'heatingType_mapping', 'kitchenType_mapping', 'facadecount_mapping',
       'latitude', 'longitude', 'primaryEnergyConsumptionPerSqm',
       'cadastralIncome'],

"""user_dict = {
    'id' : np.nan,
    'url' : np.nan,
    'type' : 'HOUSE',
    'subtype' : 'HOUSE',
    'bedroomCount' : 2,
    'bathroomCount' : 2, 
    'province' : 'Liège', 
    'locality' : 'Liège',
    'postCode' : 4041,
    'habitableSurface' : 350, 
    'roomCount' : np.nan,
    'monthlyCost' : np.nan,
    'hasAttic' : 'False',
    'hasBasement' : 'False',
    'hasDressingRoom' : np.nan,
    'diningRoomSurface' : np.nan,
    'hasDiningRoom' : 'False',
    'buildingCondition' : 'AS_NEW',
    'buildingConstructionYear' : 1899,
    'facedeCount' : 10,
    'floorCount' : np.nan,
    'streetFacadeWidth' : np.nan,
    'hasLift' : 'False',
    'floodZoneType' : np.nan, 
    'heatingType' : 'GAS',
    'hasHeatPump' : 'False',
    'hasPhotovoltaicPanels' : 'True',
    'hasThermicPanels' : np.nan,
    'kitchenSurface' : np.nan,
    'kitchenType' : 'INSTALLED',
    'landSurface' : 100,
    'hasLivingRoom' : 'False',
    'livingRoomSurface' : np.nan,
    'hasBalcony' : 'False',
    'hasGarden' : 'False',
    'gardenSurface' : 0,
    'gardenOrientation' : np.nan,
    'parkingCountIndoor' : 0,
    'parkingCountOutdoor' : np.nan,
    'hasAirConditioning' : 'False',
    'hasArmoredDoor' : np.nan,
    'hasVisiophone' : np.nan,
    'hasOffice' : 'True',
    'toiletCount' : 3,
    'hasSwimmingPool' : 'False',
    'hasFireplace' : np.nan,
    'hasTerrace' : 'False',
    'terraceSurface' : np.nan,
    'terraceOrientation' : np.nan,
    'accessibleDisabledPeople' : np.nan,
    'epcScore' : 'A',
    'price' : np.nan

}"""